### Scraping MTA status archive using selenium

Scraping historical MTA alert archive using selenium, inspired by:

https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72

In [16]:
# relative MTADelayPredict Project
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(os.path.join('AlertScraper.ipynb')))))
from MTADelayPredict.utils import scrape_helpers

In [1]:
import progressbar

In [17]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 

from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import chromedriver_binary  # Adds chromedriver binary to path
import time
import pandas as pd
import re
import os


In [18]:
from importlib import reload

In [19]:
# need to add headless and dev shm usage
# https://github.com/heroku/heroku-buildpack-google-chrome/issues/46

option = webdriver.ChromeOptions()
option.add_argument("--incognito")
option.add_argument("--disable-dev-shm-usage")
option.add_argument("--no-sandbox")
option.add_argument("--headless")

In [20]:
browser = webdriver.Chrome(executable_path='/usr/bin/chromedriver',
                           options=option)

In [21]:
browser.get('https://m.mymtaalerts.com/archive')

### Iterate through all the data entries by clicking "next" on the MTA webform

In [ ]:
download_start = '2019-01-01'
download_end = '2019-12-31'
drange = pd.date_range(start=download_start, end=download_end, freq='M')
data_dir = os.path.join('../data/raw/alerts')

for i,m in enumerate(drange):
    start_date = m.replace(day=1).strftime('%m/%d/%Y')
    end_date = m.strftime('%m/%d/%Y')
    
    browser = webdriver.Chrome(executable_path='/usr/bin/chromedriver',
                           options=option)
    browser.get('https://m.mymtaalerts.com/archive')
    
    scrape_helpers.submit_dates(browser, start_date, end_date)
    data_rows = scrape_helpers.scrape_data(browser)

    # raw send monthly csv to disk
    data_df = pd.DataFrame(data_rows, columns=['Date','Agency','Subject','Message'])
    data_df.index = data_df.Date.map(pd.to_datetime)
    #data_df['Type'] = data_df.Subject.map(lambda x:x.split(',')[-1])
    data_df.drop(columns=['Date'], inplace=True)
    
    filename = 'raw_alerts_{}_{}.csv'.format(start_date, end_date)
    filename = filename.replace('/', '.')
    data_df.to_csv(os.path.join(data_dir, filename))
    browser.quit()
    
    time.sleep(60)
    
    

 64%|######################################                      |date: 1/12/19

In [ ]:
browser.quit()

### Generate Features
#### TODO: Move to Feature Selection/Creation Notebook

In [ ]:
import progressbar
import sys
import time

In [ ]:
widgets = [progressbar.Percentage(), progressbar.Bar(), progressbar.Variable('date')]
bar = progressbar.ProgressBar(widgets=widgets, max_value=10, ).start()
for i in range(10):
    # do something
    time.sleep(0.1)
    
    bar.update(i + 1, date)
bar.finish()

In [ ]:
data_df = pd.DataFrame(data_rows, columns=['','Date','Agency','Subject','Message'])
data_df.index = data_df.Date.map(pd.to_datetime)
data_df['Type'] = data_df.Subject.map(lambda x:x.split(',')[-1])
#data_df['Trains'] = 
data_df.drop(columns=['Date'], inplace=True)

In [ ]:
status_regex = r'^(?P<Update>UPDATED:)? ?(?P<Borough>[A-Z]+), (?P<Trains>.+?) Trains?, (?P<Type>.*).*$'
status = data_df.Subject.str.extract(status_regex, expand=True)
status['Update'] = status['Update'] == 'UPDATED:'
status['Trains'] = status['Trains'].str.replace(r'[^A-Z0-9]','')

In [ ]:
status

In [552]:
data_df = data_df.join(status)

In [558]:
data_dir = 'data'
data_df.to_csv(os.path.join(data_dir, 'mta_{}_{}.csv'.format(download_start, download_end)))

In [501]:
r = re.compile(r'^(?P<update>UPDATED: )?(?P<borough>[A-Z]+), (?P<trains>.+?) Trains?, (?P<type>.*).*$')
r.match(data_df.Subject[10]).groups()

('UPDATED: ', 'BKLYN', 'G', 'Mechanical Problems')

In [7]:
browser.quit()

In [559]:
data_df

Agency  \
Date                            
2014-04-17 01:59:00       NYC   
2014-04-17 02:05:00       NYC   
2014-04-17 02:14:00       NYC   
2014-04-17 02:53:00       NYC   
2014-04-17 03:03:00       NYC   
...                 ..    ...   
2014-04-17 19:45:00       NYC   
2014-04-17 19:49:00       NYC   
2014-04-17 19:58:00       NYC   
2014-04-17 20:14:00       NYC   
2014-04-17 20:18:00       NYC   

                                                               Subject  \
Date                                                                     
2014-04-17 01:59:00                    BKLYN, 2 Trains, Rail Condition   
2014-04-17 02:05:00           UPDATED: BKLYN, 2 Trains, Rail Condition   
2014-04-17 02:14:00          SI, Staten Island Railroad, FDNY Activity   
2014-04-17 02:53:00                 MANH, A & E Trains, Rail Condition   
2014-04-17 03:03:00        UPDATED: MANH, A & E Trains, Rail Condition   
...                                                                ...   
2014-04-17 19:45:00            UPDATED: BKLYN, L Trains, Sick Customer   
2014-04-17 19:49:00                MANH, 2 Trains, Mechanical Problems   
2014-04-17 19:58:00       UPDATED: MANH, 2 Trains, Mechanical Problems   
2014-04-17 20:14:00  UPDATED: MANH, 2 and 3 Trains, Mechanical Prob...   
2014-04-17 20:18:00       UPDATED: MANH, R Trains, Mechanical Problems   

                                                               Message  \
Date                                                                     
2014-04-17 01:59:00  s/b 2 trains delayed, due to a rail condition ...   
2014-04-17 02:05:00  Following an earlier incident at Church Av, 2 ...   
2014-04-17 02:14:00  b/d no SIR service b/t Oakwood Hts and Eltingv...   
2014-04-17 02:53:00  n/b A trains running exp from Canal St to 59 S...   
2014-04-17 03:03:00  n/b A trains running exp from Canal St to 59 S...   
...                                                                ...   
2014-04-17 19:45:00  Following an earlier incident at Atlantic Av, ...   
2014-04-17 19:49:00  s/b, 2 trains via 5 from 149 St to Nevins St, ...   
2014-04-17 19:58:00  some s/b, 2 via 5 from 149 St to Nevins St & s...   
2014-04-17 20:14:00  Following an earlier incident at Chambers St, ...   
2014-04-17 20:18:00  Following an earlier incident at Canal St, R t...   

                                     Type  Updated  update borough trains  \
Date                                                                        
2014-04-17 01:59:00        Rail Condition    False   False   BKLYN      2   
2014-04-17 02:05:00        Rail Condition     True    True   BKLYN      2   
2014-04-17 02:14:00         FDNY Activity    False   False     NaN    NaN   
2014-04-17 02:53:00        Rail Condition    False   False    MANH     AE   
2014-04-17 03:03:00        Rail Condition     True    True    MANH     AE   
...                                   ...      ...     ...     ...    ...   
2014-04-17 19:45:00         Sick Customer     True    True   BKLYN      L   
2014-04-17 19:49:00   Mechanical Problems    False   False    MANH      2   
2014-04-17 19:58:00   Mechanical Problems     True    True    MANH      2   
2014-04-17 20:14:00   Mechanical Problems     True    True    MANH     23   
2014-04-17 20:18:00   Mechanical Problems     True    True    MANH      R   

                                    type  
Date                                      
2014-04-17 01:59:00       Rail Condition  
2014-04-17 02:05:00       Rail Condition  
2014-04-17 02:14:00                  NaN  
2014-04-17 02:53:00       Rail Condition  
2014-04-17 03:03:00       Rail Condition  
...                                  ...  
2014-04-17 19:45:00        Sick Customer  
2014-04-17 19:49:00  Mechanical Problems  
2014-04-17 19:58:00  Mechanical Problems  
2014-04-17 20:14:00  Mechanical Problems  
2014-04-17 20:18:00  Mechanical Problems  

[74 rows x 10 columns]